In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB
Sat Apr 20 16:07:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              24W / 300W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/

In [3]:
!pip install farasapy==0.0.14
!pip install pyarabic==0.6.14
!git clone https://github.com/aub-mind/arabert
!pip install emoji==1.6.1
!pip install sentencepiece==0.1.96
!pip install seqeval
#!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 3.6 MB/s eta 0:00:00
Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 15.73 MiB/s, done.
Resolving deltas: 100% (339/339), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.0/170.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169297 sha256=883b16206fd1f8bddbcac97b8db5199e33152be5e031fa2179c791768c46e7d7
  Stored in directory: /root/.cache/pip/wheels/3d/c9/af/02caa5725634f27f4e2e43852f67fc9069d014038b236a827e
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.1 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Unins

In [4]:
!git clone https://github.com/AhmedAbdel-Aal/WNER_24_sharedtask.git
!cp -r WNER_24_sharedtask/code/*.py .
!rm -rf WNER_24_sharedtask

Cloning into 'WNER_24_sharedtask'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 73 (delta 25), reused 68 (delta 20), pack-reused 0
Receiving objects: 100% (73/73), 53.12 KiB | 5.90 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [5]:
import numpy as np
import torch
import random
import copy
import pandas as pd


from arabert.preprocess import ArabertPreprocessor

from torch.utils.data import DataLoader, Dataset
from transformers import (AutoConfig, AutoModel, AutoModelForSequenceClassification,
                          AutoTokenizer, BertTokenizer, Trainer,
                          TrainingArguments)
from transformers.data.processors.utils import InputFeatures

from arabert.preprocess import ArabertPreprocessor
import numpy as np

from transformers import AutoConfig, AutoModelForTokenClassification, AutoTokenizer
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
from utils import *
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from tqdm import tqdm

In [6]:
train_data = load_json('/content/drive/MyDrive/anlp24/WojoodNER_fine_Data/split70.json')
dev_data = load_json('/content/drive/MyDrive/anlp24/WojoodNER_fine_Data/split10.json')

def normalize_data(items):
    normalized_data = []
    text_lists = []
    label_list = []
    for item in items:
        sentence_id = item['global_sentence_id']
        sentence = []
        labels = []
        for token_info in item['tokens']:
            word = token_info['token']
            tv = token_info['tags'][0]
            label = tv['value']
            sentence.append(word)
            labels.append(label)
        #sentence = ' '.join(sentence)
        #labels = ' '.join(labels)
        text_lists.append(sentence)
        label_list.append(labels)
    return text_lists, label_list


# Normalize the data
text_train, labels_train = normalize_data(train_data)
text_dev, labels_dev = normalize_data(dev_data)

data loaded from path /content/drive/MyDrive/anlp24/WojoodNER_fine_Data/split70.json
data loaded from path /content/drive/MyDrive/anlp24/WojoodNER_fine_Data/split10.json


In [7]:
# prompt: load the saved model

from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir")
model_loaded = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir").to(device)


In [8]:
label_map = model_loaded.config.label2id
inv_label_map = model_loaded.config.id2label

In [9]:
from data import NERDataset

In [10]:
model_name = 'aubmindlab/bert-base-arabertv02'
task_name = 'tokenclassification'
tokenizer_name = "/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir"

In [11]:
train_dataset = NERDataset(
    texts=text_train,
    tags=labels_train,
    label_map=label_map,
    model_name=model_name,
    tokenizer_name=tokenizer_name,
    max_length=512,
    )

dev_dataset = NERDataset(
    texts=text_dev,
    tags=labels_dev,
    label_map=label_map,
    model_name=model_name,
    tokenizer_name=tokenizer_name,
    max_length=512,
    )

In [12]:
def set_seed(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic=True
  torch.backends.cudnn.benchmark = False

set_seed(25)

In [13]:
def collate_fn(batch):
    """
    Custom collate_fn function to handle batching of InputFeatures.

    Args:
    - batch: A list of InputFeatures instances.

    Returns:
    - A dictionary with keys 'input_ids', 'attention_mask', 'token_type_ids', 'labels',
      where the values are batched tensors.
    """
    # Initialize containers for the various features
    input_ids = []
    attention_masks = []
    token_type_ids = []
    labels = []
    texts = []


    for item in batch:
        #print(item)
        input_ids.append(torch.tensor(item['input_ids']))  # Convert to tensor
        attention_masks.append(torch.tensor(item['attention_mask']))  # Convert to tensor
        token_type_ids.append(torch.tensor(item['token_type_ids']))  # Convert to tensor
        labels.append(torch.tensor(item['labels']))
        texts.append(item['text'])

    # Convert lists to tensors and stack them
    input_ids = torch.stack(input_ids)
    attention_masks = torch.stack(attention_masks)
    token_type_ids = torch.stack(token_type_ids)
    labels = torch.stack(labels)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'token_type_ids': token_type_ids,
        'label': labels,
        'texts':texts
    }

# Prepare DataLoader'
#batch_size = 16  # Or any batch size that fits your memory
#train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False, num_workers=5)
#test_loader = DataLoader(dev_dataset, batch_size=batch_size,collate_fn=collate_fn, shuffle=False, num_workers=5)

In [14]:
batch_size = 16  # Or any batch size that fits your memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False, num_workers=5)
test_loader = DataLoader(dev_dataset, batch_size=batch_size,collate_fn=collate_fn, shuffle=False, num_workers=5)

# Infere - Model Only

In [15]:
def align_predictions_2(predictions, label_ids):
    predictions = predictions.detach().cpu().numpy()
    label_ids = label_ids.detach().cpu().numpy()
    preds = np.argmax(predictions, axis=2)

    batch_size, seq_len = preds.shape

    ignored_index = torch.nn.CrossEntropyLoss().ignore_index

    valid_mask = label_ids != ignored_index

    # Use boolean indexing to filter out the embeddings and logits
    filtered_preds = preds[valid_mask]
    filtered_labels = label_ids[valid_mask]

    # Map labels using inv_label_map, vectorized operation
    out_label_list = [inv_label_map[label.item()] for label in filtered_labels]
    preds_list = [inv_label_map[label.item()] for label in filtered_preds]

    assert len(preds_list) == len(out_label_list)
    return preds_list, out_label_list

In [16]:
text_tokens = []
predicted_tag_1 = []
true_tags = []
model_loaded.eval()
for batch in tqdm(test_loader):
    batch = {k: v.to(device) if 'texts' not in k else v for k, v in batch.items()}
    with torch.no_grad():
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        labels = batch['label']
        outputs = model_loaded(input_ids, attention_mask, token_type_ids)
        preds, aligned_true_tags = align_predictions_2(outputs.logits, labels)

        text_tokens.append(batch['texts'])
        predicted_tag_1.append(preds)
        true_tags.append(aligned_true_tags)

  0%|          | 0/207 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 207/207 [00:33<00:00,  6.11it/s]


In [17]:
assert len(predicted_tag_1) == len(true_tags)

In [24]:
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from seqeval.scheme import IOB2
r = {
  'acc:       ': accuracy_score(true_tags, predicted_tag_1),
  "micro_f1": f1_score(true_tags, predicted_tag_1, average="micro", scheme=IOB2),
  "macro_f1": f1_score(true_tags, predicted_tag_1, average="macro", scheme=IOB2),
  "weights_f1": f1_score(true_tags, predicted_tag_1, average="weighted", scheme=IOB2),
  "precision": precision_score(true_tags, predicted_tag_1, scheme=IOB2),
  "recall": recall_score(true_tags, predicted_tag_1, scheme=IOB2)}
r

{'acc:       ': 0.9729612316888804,
 'micro_f1': 0.9076582803428257,
 'macro_f1': 0.7357219022042203,
 'weights_f1': 0.908298002364155,
 'precision': 0.8967495219885278,
 'recall': 0.9188357122865939}

# KNN

In [144]:
def create_index(embeddings, label_ids):
    batch_size, seq_len, _ = embeddings.shape
    label_ids = label_ids.detach().cpu().numpy()
    embeddings = embeddings.detach().cpu().numpy()

    ignored_index = torch.nn.CrossEntropyLoss().ignore_index


    valid_mask = label_ids != ignored_index

    # Use boolean indexing to filter out the embeddings and logits
    filtered_embeddings = embeddings[valid_mask]
    filtered_labels = label_ids[valid_mask]

    # Map labels using inv_label_map, vectorized operation
    out_label_list = [inv_label_map[label.item()] for label in filtered_labels]

    assert len(filtered_embeddings) == len(filtered_labels)
    return torch.tensor(filtered_embeddings), out_label_list

In [145]:
def create_datastore(model, loader, device):
    train_embeddings = []
    train_labels = []

    model.eval()
    for batch in tqdm(loader):
        batch = {k: v.to(device) if 'texts' not in k else v for k, v in batch.items()}
        with torch.no_grad():
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            token_type_ids = batch['token_type_ids']
            labels = batch['label']
            outputs = model(input_ids, attention_mask, token_type_ids,output_hidden_states=True)
            embeddings = outputs.hidden_states[-1]

            in_embeddings, in_labels = create_index(embeddings, labels)
            train_embeddings.extend(in_embeddings)
            train_labels.extend(in_labels)
    return np.vstack(train_embeddings), np.vstack(train_labels)

In [21]:
datastore_keys, datastore_values = create_datastore(model_loaded, train_loader, device)

  0%|          | 0/1446 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 1446/1446 [03:50<00:00,  6.26it/s]


In [22]:
datastore_keys.shape, datastore_values.shape

((390900, 768), (390900, 1))

In [23]:
# prompt: save datastore_keys and datastore_values as .npy files

import numpy as np
np.save('/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir2/datastore_keys.npy', datastore_keys)
np.save('/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir2/datastore_values.npy', datastore_values)


In [146]:
# prompt: load datastore_keys and values from .npy files

import numpy as np
datastore_keys = np.load('/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir/datastore_keys.npy')
datastore_values = np.load('/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir/datastore_values.npy')


In [157]:
datastore_values_mapped = torch.tensor(datastore_values)
datastore_values_mapped.shape

torch.Size([390900])

In [128]:
datastore_values = datastore_values.reshape(-1)
datastore_values_mapped = [label_map[x] for x in datastore_values]
datastore_values_mapped = torch.tensor(datastore_values_mapped)
datastore_values_mapped.shape

KeyError: 0

In [148]:
keys = torch.from_numpy(datastore_keys.transpose(1, 0))
keys.shape

torch.Size([768, 390900])

In [149]:
def choose(embeddings, label_ids, logits, inv_label_map):
    batch_size, seq_len, _ = embeddings.shape
    label_ids = label_ids.detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    embeddings = embeddings.detach().cpu().numpy()

    ignored_index = torch.nn.CrossEntropyLoss().ignore_index

    valid_mask = label_ids != ignored_index

    # Use boolean indexing to filter out the embeddings and logits
    filtered_embeddings = embeddings[valid_mask]
    filtered_logits = logits[valid_mask]
    filtered_labels = label_ids[valid_mask]

    # Map labels using inv_label_map, vectorized operation
    out_label_list = [inv_label_map[label.item()] for label in filtered_labels]

    assert len(filtered_embeddings) == len(filtered_logits)
    assert len(filtered_embeddings) == len(filtered_labels)
    return torch.tensor(filtered_embeddings), out_label_list, torch.tensor(filtered_logits)

In [158]:
def postprocess_logits_to_labels(logits, embedding_test, keys, datastore_values_mapped):
  #i = 2
  batch = embedding_test.shape[0]
  num_labels=42
  hidden_size = embedding_test.shape[-1]
  ##########
  embedding_test = embedding_test.view(-1, hidden_size)
  ##########
  #print(embedding_test.shape)

  sim = torch.mm(embedding_test, keys)

  #print(sim.shape)

  #norm_1 = (keys ** 2).sum(dim=0, keepdim=True).sqrt()
  #norm_2 = (embedding_test ** 2).sum(dim=1, keepdim=True).sqrt()

  norm_keys = torch.norm(keys, dim=0, keepdim=True)
  norm_embeddings = torch.norm(embedding_test, dim=1, keepdim=True)



  #scores = (sim / (norm_1 + 1e-10) / (norm_2 + 1e-10)).view(1, embedding_test.shape[0], -1)
  scores = sim / (norm_keys * norm_embeddings + 1e-10)
  scores = scores.view(1, embedding_test.shape[0], -1)

  #print(scores.shape)

  topk_scores, topk_idxs = torch.topk(scores, dim=-1, k=512)
  #print(topk_idxs[:,i])
  #print(topk_scores[:,i])

  datastore_size = keys.shape[1]
  #print(datastore_size)
  knn_labels = datastore_values_mapped.unsqueeze(0)
  #print(knn_labels)
  knn_labels = knn_labels.view(1, 1, datastore_size).expand(1, batch, datastore_size)
  knn_labels = knn_labels.gather(dim=-1, index=topk_idxs)
  #print(knn_labels[:,i])

  sim_probs = torch.softmax(topk_scores/0.5 , dim=-1)
  #print(sim_probs[:,i])
  knn_probabilities = torch.zeros_like(sim_probs[:, :, 0]).unsqueeze(-1).repeat([1,1,num_labels])
  knn_probabilities = knn_probabilities.scatter_add(dim=2, index=knn_labels, src=sim_probs)
  knn_probabilities = knn_probabilities.squeeze(0)

  return knn_probabilities, logits

In [159]:
def knn_inference(model, device, loader, keys, values_mapped):
    text_tokens = []
    all_data_logits = []
    all_knn_logits = []
    all_labels = []

    model.eval()
    for batch in tqdm(loader):
        batch = {k: v.to(device) if 'texts' not in k else v for k, v in batch.items()}
        with torch.no_grad():
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            token_type_ids = batch['token_type_ids']
            labels = batch['label']
            outputs = model(input_ids, attention_mask, token_type_ids,output_hidden_states=True)

            test_embeddings = outputs.hidden_states[-1]
            test_logits = outputs.logits
            test_labels = labels

            aligned_embeddings, aligned_labels, aligned_logits = choose(test_embeddings, test_labels, test_logits, inv_label_map)

            knn_logits, logits = postprocess_logits_to_labels(aligned_logits, aligned_embeddings, keys, datastore_values_mapped)

            text_tokens.append(batch['texts'])
            all_data_logits.append(logits)
            all_knn_logits.append(knn_logits)
            all_labels.append(aligned_labels)
    return text_tokens, all_data_logits, all_knn_logits, all_labels

In [160]:
text_tokens, all_data_logits, all_knn_logits, all_labels = knn_inference(model_loaded, device, test_loader, keys, datastore_values_mapped)

  0%|          | 0/207 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 207/207 [04:21<00:00,  1.26s/it]


In [161]:
len(text_tokens), len(all_data_logits), len(all_knn_logits), len(all_labels)

(207, 207, 207, 207)

In [163]:
lambda_ = 0.0

for lambda_ in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
  predicted_tag = []
  true_tag = []
  for i in range(len(all_data_logits)):
    data_logtis = all_data_logits[i].squeeze(0)
    knn_logits = all_knn_logits[i].squeeze(0)

    probabilities = lambda_*data_logtis + (1-lambda_)*knn_logits

    argmax_labels = torch.argmax(probabilities, -1, keepdim=False)
    argmax_labels = argmax_labels.detach().cpu().numpy()

    predicted_tag.append([inv_label_map[label] for label in argmax_labels])
    true_tag.append(all_labels[i])

  print('f1:        ',f1_score(true_tag, predicted_tag))
  #print('acc:       ', accuracy_score(true_tag, predicted_tag))
  #print('precision: ', precision_score(true_tag, predicted_tag))
  #print('recall:    ', recall_score(true_tag, predicted_tag))
  print('----------------------------------------')


f1:         0.9022711631108054
----------------------------------------
f1:         0.9054993461353155
----------------------------------------
f1:         0.9060583086360191
----------------------------------------
f1:         0.9066960899248327
----------------------------------------
f1:         0.9067825846960601
----------------------------------------
f1:         0.9077475486811214
----------------------------------------
f1:         0.9082619508151423
----------------------------------------
f1:         0.9080610623748013
----------------------------------------
f1:         0.9083120292959304
----------------------------------------
f1:         0.9080102287649459
----------------------------------------
f1:         0.9076582803428257
----------------------------------------
